## Counting objects with YOLO8

In this exercise, we will count a particular object in real-time using [YOLOv8](https://docs.ultralytics.com/models/yolov8/) object detection model. 

In [ ]:
%pip install -q opencv-python pyyaml ultralytics paho-mqtt

In [ ]:
import platform, cv2, time, collections, torch, yaml, json
from ultralytics import YOLO
from pathlib import Path
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
from pprint import pprint

# Define the model to be used.
det_model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different model sizes
    
# Loading the model names.
label_map = det_model.model.names
reversed_label_map = {v: k for k, v in label_map.items()}

# Need to make en empty call to initialize the model
res = det_model()


### Detect and count objects
Configure your choice of video and object to count by adjusting the variables below.

Possible object types can be found by reviewing [coco.yaml](../artifacts/coco.yaml)

Number of counted objects will be written to a json file.

In [ ]:
# Sample videos: 
#
# https://download.microsoft.com/download/caaf80b6-2394-4fbc-8430-8b41a3206c64/people-are-pushing-carts-along.mp4
# https://download.microsoft.com/download/a0ac5d61-60b6-4037-9555-ba5acefeb0c8/people-near-shop-counter-fruit.mp4
sourceVideo = "https://download.microsoft.com/download/caaf80b6-2394-4fbc-8430-8b41a3206c64/people-are-pushing-carts-along.mp4"
objectToCount = "person" # You can change this to any object in the model - backpack, handbag, suitcase, apple, orange, giraffe

# Initialize a dictionary to store the counts
object_counts = {}

# Modify the run_inference function to update the object_counts dictionary
def run_inference(source, deviceType, objectToCount):
    objectid = reversed_label_map[objectToCount]
    frame_count = 0
    cap = cv2.VideoCapture(source)
    assert cap.isOpened(), "Error reading video file"
    
    line_points = [(0, 600), (800, 600),(800, 0),(0,0),(0,600)]  # line or region points

    # open the video feed
    while cap.isOpened():
        success, frame = cap.read()
        object_count = 0
        if not success:
            print("Video frame is empty or video processing has been successfully completed.")
            break
        # Count persons in the current frame
        results = det_model(frame)
        for result in results:
            for box in result.boxes:
                if box.cls[0] == objectid:
                    object_count += 1
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'{objectToCount}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
         
        # Update the object_counts dictionary
        object_counts[frame_count] = object_count
        frame_count += 1

        # Display the frame with the count
        cv2.putText(frame, f'{label_map[objectid]}: {object_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Write the object_counts dictionary to a JSON file
    with open('object_counts.json', 'w') as f:
        json.dump(object_counts, f)

# Run the inference
run_inference(
    source=sourceVideo,
    deviceType="GPU",
    objectToCount=objectToCount
)


### Plot some data
Now we can plot the [data](./object_counts.json) captured by our app.

In [ ]:
# Plot the object counts data
plt.figure(figsize=(10, 5))
plt.plot(list(object_counts.keys()), list(object_counts.values()), marker='o')
plt.title(f'Count of {objectToCount} in each frame')
plt.xlabel('Frame')
plt.ylabel('Count')
plt.grid(True)
plt.show()

### Send to MQTT
By capturing data as json messages we can send them to an MQTT broker. These messages can then be used as triggers or inputs for other types of actions.


In [ ]:
import paho.mqtt.client as mqtt

# Define the MQTT broker details
broker = "test.mosquitto.org"
port = 1883
topic = "Ignite2024/Labs/ObjectCounts"

# Create an MQTT client instance
client = mqtt.Client()

# Connect to the broker
client.connect(broker, port, 60)

# Publish the object counts to the MQTT broker
for frame, count in object_counts.items():
    payload = json.dumps({"frame": frame, "count": count})
    client.publish(topic, payload)
    print(f"Published: {payload}")

# Disconnect from the broker
client.disconnect()

## Complete Lab
Run the following cell to complete this lab!

In [ ]:
%store -r userId
import requests;print(requests.post("https://jsleaderboard001-cnece0effvapgbft.westus2-01.azurewebsites.net/complete_task", headers={"Content-Type": "application/json"}, json={"user_id": userId, "task_id": 3}).json())